## load libs

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from process_images import *
from pystripe.core import *
import matplotlib.pyplot as plt
def plot_images(img_list: List[ndarray], img_labels: List[str], vmax: int):
    if len(img_list) == 1:
        plt.figure(figsize=(20, 20))
        plt.imshow(img_list[0], cmap='gray', vmin=0, vmax=vmax)
        plt.title(img_labels[0])
    else:
        fig, axes = plt.subplots(nrows=1, ncols=len(img_list), figsize=(20, 20))
        for idx, (im, label) in enumerate(zip(img_list, img_labels)):
            axes[idx].imshow(im, cmap='gray', vmin=0, vmax=vmax)
            axes[idx].set_title(label)
    plt.tight_layout()
    plt.show()

In [ ]:
from parallel_image_processor import *
tsv_volume = TSVVolume.load(r'E:\20230510_13_34_13_SM230308_05_LS_15x_800z_MIP_stitched\Ex_488_Em_525_MIP_xml_import_step_5.xml')
shape: Tuple[int, int, int] = tsv_volume.volume.shape  # shape is in z y x format
img = tsv_volume.imread(
    VExtent(
        tsv_volume.volume.x0, tsv_volume.volume.x1,
        tsv_volume.volume.y0, tsv_volume.volume.y1,
        tsv_volume.volume.z0 + shape[0]//2, tsv_volume.volume.z0 + shape[0]//2 + 1),
    tsv_volume.dtype)[0]
parallel_image_processor(
    source=TSVVolume.load(r'/data/20230419_17_34_03_SM221011_06_LS_15x_800z_stitched/Ex_488_Em_525_xml_import_step_5.xml'),
    destination=r"/data/20230419_17_34_03_SM221011_06_LS_15x_800z_stitched/Ex_488_Em_525_tif",
    fun=process_img,
    kwargs={'bleach_correction_frequency': 0.0005, 'bleach_correction_max_method': False, 'bleach_correction_y_slice_max': None, 'threshold': None, 'sigma': (4000.0, 4000.0), 'bidirectional': True, 'lightsheet': False, 'percentile': 0.25, 'rotate': 90, 'convert_to_8bit': False, 'bit_shift_to_right': 8, 'tile_size': (39220, 28056), 'd_type': 'uint16', "verbose": True},
    source_voxel=(0.8, 0.4, 0.4),
    target_voxel=20,
    max_processors=1
)

In [ ]:
def get_layer(
    index: int,          # layer of image requested
    image: ndarray,      # 3-D image (use TifStack.as_3d_numpy())
    plane = "xy",        # must be "xy", "yx", "xz", "zx", "yz", "zy"
    img_format = "zyx",  # xyz in some order
):
    # guards
    if plane not in {"xy", "yx", "xz", "zx", "yz", "zy"} or img_format not in {"zyx", "zxy", "yxz", "yzx", "xyz", "xzy"}:
        print(f"Invalid plane selected in get_layer().  Plane: {plane}, Layer: {index}, Img_format: {img_format}\nReturning to caller...")
        return None

    # get the layer
    if 'x' not in plane:   sub = img_format.index('x')
    elif 'y' not in plane: sub = img_format.index('y')
    elif 'z' not in plane: sub = img_format.index('z')

    if sub == 0:   layer_image = image[index, :, :]
    elif sub == 1: layer_image = image[:, index, :]
    elif sub == 2: layer_image = image[:, :, index]

    # if plane is flipped compared to image format, return the transpose.
    if plane not in (img_format[:sub] + img_format[sub + 1:]):
        return layer_image.transpose()
    return layer_image

# run to rotate images
from os import makedirs

######## CHANGE THESE ########
cha1_path = "D:/BMAP/Brain 4/cha1"
cha2_path = "D:/BMAP/Brain 4/cha2"
cha3_path = "D:/BMAP/Brain 4/cha3"
# set to None if nothing to convert

##############################
if cha1_path:
    makedirs(cha1_path + "_zx", exist_ok=True)
    makedirs(cha1_path + "_zy", exist_ok=True)
    stack1 = TifStack(cha1_path).as_3d_numpy()
    for i in range(stack1.shape[1]):
        imwrite(cha1_path + "_zx/" + str(i + 1) + ".tif", get_layer(i, stack1, "zx"))
    for i in range(stack1.shape[2]):
        imwrite(cha1_path + "_zy/" + str(i + 1) + ".tif", get_layer(i, stack1, "zy"))
if cha2_path:
    makedirs(cha2_path + "_zx", exist_ok=True)
    makedirs(cha2_path + "_zy", exist_ok=True)
    stack2 = TifStack(cha2_path).as_3d_numpy()
    for i in range(stack2.shape[1]):
        imwrite(cha2_path + "_zx/" + str(i + 1) + ".tif", get_layer(i, stack2, "zx"))
    for i in range(stack2.shape[2]):
        imwrite(cha2_path + "_zy/" + str(i + 1) + ".tif", get_layer(i, stack2, "zy"))
if cha3_path:
    makedirs(cha3_path + "_zx", exist_ok=True)
    makedirs(cha3_path + "_zy", exist_ok=True)
    stack3 = TifStack(cha3_path).as_3d_numpy()
    for i in range(stack3.shape[1]):
        imwrite(cha3_path + "_zx/" + str(i + 1) + ".tif", get_layer(i, stack3, "zx"))
    for i in range(stack3.shape[2]):
        imwrite(cha3_path + "_zy/" + str(i + 1) + ".tif", get_layer(i, stack3, "zy"))
print("Operation Completed")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_data(file_path:str, title:str, label1:str, label2:str):
    with open(file_path, "r") as file:
        line = file.readline()

        x_offset = [0]
        y_offset = [0]

        while line:
            if "Skipped" in line:
                x_offset.append(0)
                y_offset.append(0)
                line = file.readline()
                continue
            s = line.split(",")
            x_offset.append(float(s[-2][:-1]))
            line = file.readline()
            s = line.split(",")
            y_offset.append(float(s[-2][:-1]))
            line = file.readline()
            # skip last line
            line = file.readline()

        plt.plot(range(len(x_offset)), x_offset, label=label1)
        plt.plot(range(len(y_offset)), y_offset, label=label2)
        plt.title(title)
        plt.xlabel("Layer index")
        plt.ylabel("Units")
        plt.ylim((-20, 20))
        plt.legend()
        plt.show()

plot_data("D:/BMAP/Brain 4/offsets/zy_matrices_im12.txt", "Image alignment offsets of Images 1 and 2 from zy-slices", "z-offset", "y-offset")
plot_data("D:/BMAP/Brain 4/offsets/zy_matrices_im13.txt", "Image alignment offsets of Images 1 and 3 from zy-slices", "z-offset", "y-offset")
plot_data("D:/BMAP/Brain 4/offsets/zx_matrices_im12.txt", "Image alignment offsets of Images 1 and 2 from zx-slices", "z-offset", "x-offset")
plot_data("D:/BMAP/Brain 4/offsets/zx_matrices_im13.txt", "Image alignment offsets of Images 1 and 3 from zx-slices", "z-offset", "x-offset")
plot_data("D:/BMAP/Brain 4/offsets/xy_matrices_im12.txt", "Image alignment offsets of Images 1 and 2 from xy-slices", "x-offset", "y-offset")
plot_data("D:/BMAP/Brain 4/offsets/xy_matrices_im13.txt", "Image alignment offsets of Images 1 and 3 from xy-slices", "x-offset", "y-offset")

In [18]:
from pathlib import Path
from align_images import get_layer
from numpy import min, max, uint8, zeros_like, ndarray, multiply

def write_to_file(images: list[ndarray], filepath: Path):
    filepath.mkdir(parents=True, exist_ok=True)
    for n, image in enumerate(images):
        local = filepath / f'cha{n}'
        local.mkdir(parents=True, exist_ok=True)
        for layer in range(image.shape[0]):
            path = local.absolute() / (str(layer) + ".tif")
            imwrite(path, get_layer(layer, image, "yx"))
        print("wrote to file")
        
# written by ChatGPT
def normalize_array_inplace(arr: ndarray):
    min_val = min(arr)
    max_val = max(arr)

    arr -= min_val
    arr /= (max_val - min_val)

    # Scale the values to be between 0 and 255
    arr *= 255
    arr.astype(uint8, copy=False)
    

# multiplies two 2d-ndarrays, saving solution in arr1 instead of allocating more memory
def mult_in_place(arr1, arr2):
    for r in range(arr1.shape[0]):
        for c in range(arr1.shape[1]):
            arr1[r][c] *= arr2[r][c]
 
 
def get_borders(img: ndarray, copy=False):
    print("get_borders")
    mask = zeros_like(img)
    for ind in range(img.shape[0]):
        print(f'layer {ind}')
        mask[ind] = get_img_mask(img[ind], otsu_threshold(img[ind]))
    multiply(img, mask, out=img)
    
    

In [22]:
from align_images import align_all_images, resize_arrays
from supplements.tifstack import TifStack

cha1 = TifStack("Y:/3D_stitched_LS/20230825_SM230601_06_LS_15x_800z_B6/Downsampled/Ex_488_Ch0_z20.0_yx20.0um").as_3d_numpy()
cha2 = TifStack("Y:/3D_stitched_LS/20230825_SM230601_06_LS_15x_800z_B6/Downsampled/Ex_561_Ch1_z20.0_yx20.0um").as_3d_numpy()
cha3 = TifStack("Y:/3D_stitched_LS/20230825_SM230601_06_LS_15x_800z_B6/Downsampled/Ex_642_Ch2_z20.0_yx20.0um").as_3d_numpy()
output_path = Path("D:/aligned_images/20230825_SM230601_06_LS_15x_800z_B6/mask")
max_iterations = 50

print("Images loaded")

# make arrays the same size
channels = resize_arrays([cha1, cha2, cha3])

print("Images resized")
# print("test")
# for i in channels: print(i.shape)


for img in channels:
    get_borders(img)    

print("got borders")

# align images
# alignments, residuals = align_all_images(channels, verbose=True, make_copy=False)

# normalize images and convert to uint8
for channel in channels: normalize_array_inplace(channel)

print("Images normalized")

write_to_file(channels, output_path)

print("Done!")
# print("Alignment: ", end='')
# print(alignments)
# print("Residuals: ", end='')
# print(residuals) 

Images loaded
Images resized
get_borders
layer 0
layer 1
layer 2
layer 3
layer 4
layer 5
layer 6
layer 7
layer 8
layer 9
layer 10
layer 11
layer 12
layer 13
layer 14
layer 15
layer 16
layer 17
layer 18
layer 19
layer 20
layer 21
layer 22
layer 23
layer 24
layer 25
layer 26
layer 27
layer 28
layer 29
layer 30
layer 31
layer 32
layer 33
layer 34
layer 35
layer 36
layer 37
layer 38
layer 39
layer 40
layer 41
layer 42
layer 43
layer 44
layer 45
layer 46
layer 47
layer 48
layer 49
layer 50
layer 51
layer 52
layer 53
layer 54
layer 55
layer 56
layer 57
layer 58
layer 59
layer 60
layer 61
layer 62
layer 63
layer 64
layer 65
layer 66
layer 67
layer 68
layer 69
layer 70
layer 71
layer 72
layer 73
layer 74
layer 75
layer 76
layer 77
layer 78
layer 79
layer 80
layer 81
layer 82
layer 83
layer 84
layer 85
layer 86
layer 87
layer 88
layer 89
layer 90
layer 91
layer 92
layer 93
layer 94
layer 95
layer 96
layer 97
layer 98
layer 99
layer 100
layer 101
layer 102
layer 103
layer 104
layer 105
layer 106

* try using mask for better edge detection
* Sobel operator -> mask -> align

In [10]:
# convert stacks to .ims
import os
dir_name = "D:/aligned_images"

os.system(f'python convert.py -i "{dir_name}/cha0" -o "{dir_name}/cha0.ims" -dx 10 -dy 10 -dz 10')
os.system(f'python convert.py -i "{dir_name}/cha1" -o "{dir_name}/cha1.ims" -dx 10 -dy 10 -dz 10')
os.system(f'python convert.py -i "{dir_name}/cha2" -o "{dir_name}/cha2.ims" -dx 10 -dy 10 -dz 10')

0

In [39]:
!python align_images.py D:\aligned_images\20230825_SM230601_06_LS_15x_800z_B6\orig D:\aligned_images\20230825_SM230601_06_LS_15x_800z_B6\orig2
!echo done

Loading images...
Images loaded
['D:\\aligned_images\\20230825_SM230601_06_LS_15x_800z_B6\\orig\\cha0', 'D:\\aligned_images\\20230825_SM230601_06_LS_15x_800z_B6\\orig\\cha1', 'D:\\aligned_images\\20230825_SM230601_06_LS_15x_800z_B6\\orig\\cha2']
Resizing images...
Images resized
Normalizing images
Aligning images... (this may take a while)
threshold 0.03580609
threshold 0.055491004
threshold 0.061017282
threshold 12.411819
threshold 8.727011
threshold 7.828353
threshold 6.400338
threshold 5.7560625
threshold 5.6221843
threshold 4.7072563
threshold 3.9848094
threshold 3.680725
threshold 0.74060315
threshold 0.5096826
threshold 3.6868677
threshold 3.6830204
threshold 4.1994777
threshold 0.9222317
threshold 3.3971043
threshold 1.0497518
threshold 3.4798136
threshold 3.7975388
threshold 3.3997142
threshold 1.0741214
threshold 3.395099
threshold 0.830176
threshold 1.004323
threshold 1.0248448
threshold 0.9898925
threshold 1.0831723
threshold 1.0534685
threshold 3.3944578
threshold 1.2454544